# Create meeting minutes from an Audio file

I downloaded some Denver City Council meeting minutes and selected a portion of the meeting for us to transcribe. You can download it here:  
https://drive.google.com/file/d/1N_kpSojRR5RYzupz6nqM8hMSoEF_R7pU/view?usp=sharing

If you'd rather work with the original data, the HuggingFace dataset is [here](https://huggingface.co/datasets/huuuyeah/meetingbank) and the audio can be downloaded [here](https://huggingface.co/datasets/huuuyeah/MeetingBank_Audio/tree/main).

The goal of this product is to use the Audio to generate meeting minutes, including actions.

For this project, you can either use the Denver meeting minutes, or you can record something of your own!


In [ ]:
# imports
import os
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from huggingface_hub import login
from dotenv import load_dotenv
import os
from google import genai

In [31]:
# Constants

LLAMA =  os.environ.get("HF_MODEL", "meta-llama/Llama-3.2-3B-Instruct")

In [30]:
audio_filename = "D:/LLM/Projects/LLM-Meeting-Minutes-Generation/audio/denver_extract.mp3"

In [32]:
load_dotenv(override=True)
hf_api_key=os.getenv("HF_TOKEN")
hf_client = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=hf_api_key,  # set this as a Space Secret
)

#######Genai#########
genai_api_key=os.getenv('GOOGLE_API_KEY')
genai_client = genai.Client(api_key=genai_api_key)

In [33]:
login(hf_api_key, add_to_git_credential=True)

# Open the file

audio_file =  genai_client.files.upload(file=audio_filename)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


### STEP 1: Transcribe Audio

In [34]:
trascription_prompt = """Generate a verbatim transcript of this audio.
- Preserve the speaker turns if possible (Speaker 1, Speaker 2, etc. or names of speackers)
- Add punctuation and paragraph breaks
"""

In [ ]:
def get_transcription(audio_path):
    audio_file =  genai_client.files.upload(file=audio_path)
    resp = genai_client.models.generate_content(
    model="gemini-3-flash-preview",
    contents=[trascription_prompt, audio_file],)
    transcription = resp.text
    return transcription



In [36]:
transcription=get_transcription(audio_filename)

### STEP 2: Analyze & Report

In [40]:
def get_messages(transcription):
      system_message = """
      You produce minutes of meetings from transcripts, with summary, key discussion points,
      takeaways and action items with owners, in markdown format without code blocks.
      """

      user_prompt = f"""
      Below is an extract transcript of a Denver council meeting.
      Please write minutes in markdown without code blocks, including:
      - a summary with attendees, location and date
      - discussion points
      - takeaways
      - action items with owners

      Transcription:
      {transcription}
      """

      messages = [
          {"role": "system", "content": system_message},
          {"role": "user", "content": user_prompt}
        ]

      return messages

In [ ]:
def generate_meeting_minutes(audio_filename):
    transcription=get_transcription(audio_filename)  
    stream = hf_client.chat.completions.create(
        model=LLAMA,
        messages=get_messages(transcription),
        stream=True
       # streanms one by one in chunks (parts)
    )    
    response = ""

    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response
   

In [46]:
meeting_minutes=generate_meeting_minutes(audio_filename)

In [28]:
import gradio as gr

In [ ]:
with gr.Blocks(title="Meeting Minutes Generator") as demo:
    gr.Markdown("# 🎙️ Meeting Minutes Generator\nDrop audio → transcript + minutes")

    audio = gr.Audio(
        sources=["upload"],
        type="filepath",
        label="Drag & drop an audio file (mp3/wav/m4a/etc.)",
    )


    btn = gr.Button("Generate Meeting minutes")

    with gr.Tab("Meeting Minutes (Markdown)"):
        minutes_out = gr.Markdown()

    btn.click(
        fn=generate_meeting_minutes,
        inputs=[audio],
        outputs=[minutes_out],
    )

demo.queue().launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.
